In [1]:
from abc import ABC, abstractmethod
import numpy as np
import pandas as pd

class Simulation(ABC):
    @abstractmethod
    def run(self):
        pass


In [2]:
class BallDropSimulationTS(Simulation):
    """
    Ball drop simulation that records velocity, acceleration, and height at each timestep.
    """
    def __init__(self, mass, radius, initial_height, air_resistance, dt=0.01):
        self.mass = mass
        self.radius = radius
        self.initial_height = initial_height
        self.air_resistance = air_resistance
        self.dt = dt

        # Physical constants
        self.g = 9.81           # gravitational acceleration (m/s^2)
        self.rho = 1.225       # air density (kg/m^3)
        self.A = np.pi * (self.radius ** 2)  # cross-sectional area
        self.Cd = self.air_resistance       # drag coefficient

    def run(self):
        # Initialize states
        t = 0.0
        v = 0.0
        h = self.initial_height

        data_rows = []

        while h > 0:
            # Calculate drag force
            # F_drag = 0.5 * Cd * rho * A * v^2
            # Net force = m*g - F_drag (assuming downward velocity is positive)
            F_drag = 0.5 * self.Cd * self.rho * self.A * (v ** 2)
            F_net = self.mass * self.g - F_drag

            # Acceleration
            a = F_net / self.mass

            # Record the current step
            data_rows.append({
                "mass": self.mass,
                "radius": self.radius,
                "air_resistance": self.air_resistance,
                "time": t,
                "velocity": v,
                "acceleration": a,
                "height": h
            })

            # Update velocity and height
            v = v + a * self.dt
            # We assume downward is positive, so height goes down by v*dt
            h = max(h - v * self.dt, 0.0)

            # Update time
            t += self.dt

        # Return as a DataFrame
        return pd.DataFrame(data_rows)


In [3]:
def generate_timeseries_dataset(num_simulations=5,
                               dt=0.01,
                               noise_type="gaussian",
                               noise_level=0.01):
    """
    Generate a dataset with (time, velocity, acceleration, height, mass, etc.)
    for multiple randomized ball-drop simulations.
    
    Parameters:
    -----------
    num_simulations : int
        Number of independent simulations to run
    dt : float
        Timestep in seconds
    noise_type : str
        Either 'gaussian' or 'uniform' to add random noise
    noise_level : float
        The fraction of the mean to use as noise scale (e.g., 0.01 for 1%)
    """

    all_data = []

    for _ in range(num_simulations):
        # Randomly pick simulation parameters
        mass = np.random.uniform(0.1, 10)             # kg
        radius = np.random.uniform(0.01, 0.5)         # meters
        initial_height = np.random.uniform(10, 100)   # meters
        air_resistance = np.random.uniform(0.1, 1.5)  # drag coefficient

        # Create the simulation
        sim = BallDropSimulationTS(mass, radius, initial_height, air_resistance, dt=dt)
        sim_data = sim.run()  # returns a DataFrame of all timesteps

        # Optionally add noise to velocity, acceleration, height
        # For each row, we add random noise based on the chosen distribution
        for col in ["velocity", "acceleration", "height"]:
            mean_vals = sim_data[col].abs().replace(0.0, 1e-8)  # handle zero mean
            if noise_type == "gaussian":
                # e.g. noise ~ N(0, noise_level * mean_value)
                noise = np.random.normal(0, noise_level * mean_vals.values)
            elif noise_type == "uniform":
                # e.g. noise ~ U(-noise_level*mean, +noise_level*mean)
                noise = np.random.uniform(
                    -noise_level * mean_vals.values,
                    +noise_level * mean_vals.values
                )
            else:
                noise = 0
            sim_data[col] += noise

        all_data.append(sim_data)

    # Concatenate all time-step data into a single DataFrame
    final_dataset = pd.concat(all_data, ignore_index=True)
    # Save to CSV
    final_dataset.to_csv("ball_drop_timeseries.csv", index=False)
    return final_dataset

# Example usage:
df = generate_timeseries_dataset(num_simulations=5, dt=0.01, noise_type="gaussian", noise_level=0.01)
print(df.head(20))
print(f"Number of rows in dataset: {len(df)}")


        mass    radius  air_resistance  time      velocity  acceleration  \
0   6.071469  0.209352        1.495762  0.00  6.482812e-11      9.897603   
1   6.071469  0.209352        1.495762  0.01  9.777301e-02      9.874832   
2   6.071469  0.209352        1.495762  0.02  1.964118e-01      9.816021   
3   6.071469  0.209352        1.495762  0.03  2.934672e-01      9.815167   
4   6.071469  0.209352        1.495762  0.04  3.888074e-01      9.662685   
5   6.071469  0.209352        1.495762  0.05  4.881738e-01      9.869918   
6   6.071469  0.209352        1.495762  0.06  5.785147e-01      9.766443   
7   6.071469  0.209352        1.495762  0.07  6.883130e-01      9.481311   
8   6.071469  0.209352        1.495762  0.08  7.892837e-01      9.860280   
9   6.071469  0.209352        1.495762  0.09  8.811267e-01      9.745858   
10  6.071469  0.209352        1.495762  0.10  9.831594e-01      9.712811   
11  6.071469  0.209352        1.495762  0.11  1.085519e+00      9.632000   
12  6.071469

In [4]:
class SimpleHarmonicMotionSim(Simulation):
    """
    Simulate a mass-spring system without damping, using the analytical solution:
      x(t) = A cos(omega t + phi),  where omega = sqrt(k/m).
    We'll record x(t), v(t), a(t) over time.
    """
    def __init__(self, mass, spring_const, amplitude, phase, dt=0.01):
        self.mass = mass           # (m)
        self.k = spring_const      # (k)
        self.A = amplitude         # (A)
        self.phi = phase           # (phi)
        self.dt = dt
        
        # Derived quantity: omega
        self.omega = np.sqrt(self.k / self.mass)

    def run(self, num_periods=3):
        """
        Generate time, displacement x, velocity v, acceleration a
        over a chosen number of periods.
        
        num_periods: how many oscillation periods we record
                     (1 period = 2*pi / omega)
        """
        period = 2.0 * np.pi / self.omega
        max_time = period * num_periods
        
        # We'll store results for each timestep
        data_rows = []
        
        t = 0.0
        while t <= max_time:
            # Analytical equations:
            x_t = self.A * np.cos(self.omega * t + self.phi)
            v_t = -self.A * self.omega * np.sin(self.omega * t + self.phi)
            a_t = -self.A * (self.omega**2) * np.cos(self.omega * t + self.phi)
            
            data_rows.append({
                "mass": self.mass,
                "spring_const": self.k,
                "amplitude": self.A,
                "phase": self.phi,
                "time": t,
                "displacement": x_t,
                "velocity": v_t,
                "acceleration": a_t
            })
            
            t += self.dt
        
        return pd.DataFrame(data_rows)

In [5]:
def generate_shm_dataset(num_simulations=5,
                         dt=0.01,
                         num_periods=3,
                         noise_type="gaussian",
                         noise_level=0.01):
    """
    Generate a dataset of time, displacement, velocity, acceleration
    for multiple random SHM simulations (no damping).
    
    Parameters
    ----------
    num_simulations : int
        How many different (m, k, A, phi) combinations to generate.
    dt : float
        Timestep for sampling the analytical solution.
    num_periods : int
        How many oscillation periods to simulate for each run.
    noise_type : str
        "gaussian" or "uniform" noise on displacement, velocity, acceleration.
    noise_level : float
        Fraction of the mean used as noise scale (e.g. 0.01 = 1%).
    """
    all_data = []

    for _ in range(num_simulations):
        # Randomize parameters
        mass = np.random.uniform(0.1, 10.0)  # m
        spring_const = np.random.uniform(1.0, 100.0)  # k
        amplitude = np.random.uniform(0.1, 5.0)  # A
        phase = np.random.uniform(0.0, 2.0 * np.pi)   # phi
        
        # Create simulation
        sim = SimpleHarmonicMotionSim(mass, spring_const, amplitude, phase, dt=dt)
        sim_data = sim.run(num_periods=num_periods)  # time-series DataFrame
        
        # Optionally add noise to displacement, velocity, acceleration
        for col in ["displacement", "velocity", "acceleration"]:
            mean_vals = sim_data[col].abs().replace(0, 1e-12)
            
            if noise_type == "gaussian":
                noise = np.random.normal(0, noise_level * mean_vals.values)
            elif noise_type == "uniform":
                noise = np.random.uniform(
                    -noise_level * mean_vals.values,
                    +noise_level * mean_vals.values
                )
            else:
                noise = 0
            sim_data[col] += noise
        
        all_data.append(sim_data)
    
    # Concatenate all into a single DataFrame
    final_dataset = pd.concat(all_data, ignore_index=True)
    # Save
    final_dataset.to_csv("shm_dataset.csv", index=False)
    return final_dataset

# Example usage:
df_shm = generate_shm_dataset(num_simulations=5, dt=0.01, num_periods=3,
                              noise_type="gaussian", noise_level=0.01)
print(df_shm.head(15))
print("Total rows:", len(df_shm))


        mass  spring_const  amplitude     phase  time  displacement  velocity  \
0   9.174416     42.363823   4.011814  5.103647  0.00      1.545775  7.797743   
1   9.174416     42.363823   4.011814  5.103647  0.01      1.615798  7.800376   
2   9.174416     42.363823   4.011814  5.103647  0.02      1.726816  7.848850   
3   9.174416     42.363823   4.011814  5.103647  0.03      1.778081  7.688742   
4   9.174416     42.363823   4.011814  5.103647  0.04      1.836691  7.732466   
5   9.174416     42.363823   4.011814  5.103647  0.05      1.895174  7.711700   
6   9.174416     42.363823   4.011814  5.103647  0.06      1.995336  7.465495   
7   9.174416     42.363823   4.011814  5.103647  0.07      2.073936  7.487139   
8   9.174416     42.363823   4.011814  5.103647  0.08      2.145611  7.375369   
9   9.174416     42.363823   4.011814  5.103647  0.09      2.210786  7.208069   
10  9.174416     42.363823   4.011814  5.103647  0.10      2.240873  7.118571   
11  9.174416     42.363823  

In [6]:
class WavePropagationSim(Simulation):
    """
    Simulate a 1D electromagnetic wave in free space with exponential decay:
      E(x,t) = E0 * exp(-alpha * t / 2) * cos(k*x - omega*t).
    """

    def __init__(self, E0, alpha, k, omega, 
                 x_max=10.0, t_max=5.0, 
                 dx=0.1, dt=0.05):
        """
        Parameters:
        -----------
        E0 : float
            Initial amplitude of the wave.
        alpha : float
            Decay rate (controls exp(-alpha * t / 2)).
        k : float
            Wave number (2*pi / wavelength).
        omega : float
            Angular frequency (2*pi * frequency).
        x_max : float
            Maximum spatial extent in x-direction.
        t_max : float
            Maximum simulation time.
        dx : float
            Spatial step size.
        dt : float
            Time step size.
        """
        self.E0 = E0
        self.alpha = alpha
        self.k = k
        self.omega = omega
        self.x_max = x_max
        self.t_max = t_max
        self.dx = dx
        self.dt = dt

    def run(self):
        """
        Evaluate the analytical solution over a grid of x and t,
        store E(x,t) in a DataFrame.
        """
        x_values = np.arange(0, self.x_max + self.dx/2, self.dx)
        t_values = np.arange(0, self.t_max + self.dt/2, self.dt)

        data_rows = []
        for t in t_values:
            # time-based decay factor
            decay_factor = np.exp(-self.alpha * t / 2.0)
            for x in x_values:
                # E(x,t) = E0 * decay_factor * cos(k*x - omega*t)
                E_xt = self.E0 * decay_factor * np.cos(self.k*x - self.omega*t)

                data_rows.append({
                    "E0": self.E0,
                    "alpha": self.alpha,
                    "k": self.k,
                    "omega": self.omega,
                    "x": x,
                    "t": t,
                    "E": E_xt
                })

        return pd.DataFrame(data_rows)


In [8]:
def generate_wave_dataset(num_simulations=5,
                         x_max=10.0, t_max=5.0,
                         dx=0.1, dt=0.05,
                         noise_type="gaussian",
                         noise_level=0.01):
    """
    Generate a dataset for the decaying wave E(x,t) = E0 * e^(-alpha*t/2) * cos(k*x - omega*t).
    
    Parameters:
    -----------
    num_simulations : int
        Number of random wave parameter sets.
    x_max : float
        Max spatial extent in x-direction.
    t_max : float
        Max time.
    dx : float
        Grid spacing in x.
    dt : float
        Time step in t.
    noise_type : str
        "gaussian" or "uniform" to add noise to E.
    noise_level : float
        Fraction of mean used as noise scale (e.g. 0.01 = 1%).
    """
    all_data = []

    for _ in range(num_simulations):
        # Randomize wave parameters
        E0 = np.random.uniform(0.5, 5.0)    # wave amplitude
        alpha = np.random.uniform(0.1, 1.0) # decay rate
        k = np.random.uniform(0.5, 3.0)     # wave number
        omega = np.random.uniform(2.0, 10.0)# angular frequency

        sim = WavePropagationSim(E0, alpha, k, omega, 
                                 x_max=x_max, t_max=t_max,
                                 dx=dx, dt=dt)
        df_sim = sim.run()  # returns DataFrame with columns: [E0, alpha, k, omega, x, t, E]

        # Optionally add noise to E
        mean_vals = df_sim["E"].abs().replace(0.0, 1e-12)
        if noise_type == "gaussian":
            noise = np.random.normal(0, noise_level * mean_vals.values)
        elif noise_type == "uniform":
            noise = np.random.uniform(-noise_level * mean_vals.values,
                                      +noise_level * mean_vals.values)
        else:
            noise = 0
        df_sim["E"] += noise

        all_data.append(df_sim)

    # Combine all simulations
    final_dataset = pd.concat(all_data, ignore_index=True)
    final_dataset.to_csv("wave_dataset.csv", index=False)
    return final_dataset

# Example usage:
df_wave = generate_wave_dataset(num_simulations=3,
                                x_max=10.0, t_max=5.0,
                                dx=0.5, dt=0.2,
                                noise_type="gaussian",
                                noise_level=0.01)
print(df_wave.head(10))
print("Total rows:", len(df_wave))


         E0     alpha         k     omega    x    t         E
0  4.719895  0.769952  1.433768  6.777697  0.0  0.0  4.757718
1  4.719895  0.769952  1.433768  6.777697  0.5  0.0  3.524814
2  4.719895  0.769952  1.433768  6.777697  1.0  0.0  0.641149
3  4.719895  0.769952  1.433768  6.777697  1.5  0.0 -2.622157
4  4.719895  0.769952  1.433768  6.777697  2.0  0.0 -4.634926
5  4.719895  0.769952  1.433768  6.777697  2.5  0.0 -4.266071
6  4.719895  0.769952  1.433768  6.777697  3.0  0.0 -1.871025
7  4.719895  0.769952  1.433768  6.777697  3.5  0.0  1.380825
8  4.719895  0.769952  1.433768  6.777697  4.0  0.0  4.006795
9  4.719895  0.769952  1.433768  6.777697  4.5  0.0  4.605342
Total rows: 1638
